In [2]:
from src.dataset_utils import Dataset
from src.model import TFModel

2023-03-17 10:11:43.730477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Prepare Dataset

In [6]:
df = sqlContext.read.parquet("hdfs://localhost:9000/youtubeParquet/*.parquet")
pd_df = df.toPandas()

In [7]:
pd_df = pd_df.rename(columns={
    '_1' : 'video_id',
    '_2' : 'description',
    '_3' : 'published_at',
    '_4' : 'channel_name',
    '_5' : 'video_title'
})
pd_df = pd_df.drop_duplicates(subset='video_id')

In [8]:
pd_df.head()

,video_id,description,published_at,channel_name,video_title
0,poZtdyC24P4,Tutorial belajar bahasa pemrograman komputer d...,2023-02-24T23:45:01Z,Dea Afrizal,Belajar Dasar Pemrograman Untuk Pemula Menggun...
1,rfscVS0vtbw,This course will give you a full introduction ...,2018-07-11T18:00:42Z,freeCodeCamp.org,Learn Python - Full Course for Beginners [Tuto...
2,xETkm9H6aaY,Yuk Belajar Python dasar di seri Tutorial Pyth...,2023-02-13T10:00:36Z,Kelas Terbuka,Belajar Python [Dasar] - 02a - Installasi Pyth...
3,_uQrJ0TkZlc,Python tutorial - Python full course for begin...,2019-02-18T15:00:08Z,Programming with Mosh,Python Tutorial - Python Full Course for Begin...
4,kqtD5dpn9C8,This Python tutorial for beginners show how to...,2020-09-16T13:00:20Z,Programming with Mosh,Python for Beginners - Learn Python in 1 Hour


In [6]:
pd_df.to_csv('data/youtube-api.csv', index=False)

# Inference

In [4]:
train = False
#prepare the inference data
print('now creating dataset'.center(60, '-'))
data = Dataset(train)
data.prepare_inference_data()

--------------------now creating dataset--------------------


In [5]:
model = TFModel()
y = model.inference(data.df)

66/66 [==============================] - 1s 14ms/step


In [17]:
pd_df['sentiment'] = y

In [18]:
pd_df.head()

,video_id,description,published_at,channel_name,video_title,sentiment
0,poZtdyC24P4,Tutorial belajar bahasa pemrograman komputer d...,2023-02-24T23:45:01Z,Dea Afrizal,Belajar Dasar Pemrograman Untuk Pemula Menggun...,0
1,rfscVS0vtbw,This course will give you a full introduction ...,2018-07-11T18:00:42Z,freeCodeCamp.org,Learn Python - Full Course for Beginners [Tuto...,1
2,xETkm9H6aaY,Yuk Belajar Python dasar di seri Tutorial Pyth...,2023-02-13T10:00:36Z,Kelas Terbuka,Belajar Python [Dasar] - 02a - Installasi Pyth...,2
3,_uQrJ0TkZlc,Python tutorial - Python full course for begin...,2019-02-18T15:00:08Z,Programming with Mosh,Python Tutorial - Python Full Course for Begin...,1
4,kqtD5dpn9C8,This Python tutorial for beginners show how to...,2020-09-16T13:00:20Z,Programming with Mosh,Python for Beginners - Learn Python in 1 Hour,1


In [11]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(pd_df) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- video_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- published_at: string (nullable = true)
 |-- channel_name: string (nullable = true)
 |-- video_title: string (nullable = true)
 |-- sentiment: long (nullable = true)

+-----------+--------------------+--------------------+--------------------+--------------------+---------+
|   video_id|         description|        published_at|        channel_name|         video_title|sentiment|
+-----------+--------------------+--------------------+--------------------+--------------------+---------+
|poZtdyC24P4|Tutorial belajar ...|2023-02-24T23:45:01Z|         Dea Afrizal|Belajar Dasar Pem...|        0|
|rfscVS0vtbw|This course will ...|2018-07-11T18:00:42Z|    freeCodeCamp.org|Learn Python - Fu...|        1|
|xETkm9H6aaY|Yuk Belajar Pytho...|2023-02-13T10:00:36Z|       Kelas Terbuka|Belajar Python [D...|        2|
|_uQrJ0TkZlc|Python tutorial -...|2019-02-18T15:00:08Z|Programming with ...|Python Tutorial -

In [13]:
sparkDF.write.save('hdfs://localhost:9000/youtube_sentiment/', format='parquet', mode='append')

23/03/17 10:16:10 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/17 10:16:10 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
[Stage 5:>                                                          (0 + 8) / 8]23/03/17 10:16:11 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [14]:
#test hasil loading inference data
load_df = sqlContext.read.parquet("hdfs://localhost:9000/youtube_sentiment/*.parquet")
load_df.show()

+-----------+--------------------+--------------------+--------------------+-------------------------+---------+
|   video_id|         description|        published_at|        channel_name|              video_title|sentiment|
+-----------+--------------------+--------------------+--------------------+-------------------------+---------+
|ZkFKF-ohsKk|Get notified of t...|2023-03-07T04:26:42Z|    Nicholas Renotte|     How to use the Ch...|        1|
|aPhbz4fER0o|★ Gabung Menjadi ...|2022-02-24T13:00:11Z|YOUKU  Indonesia-...|     INDO SUB (Snake I...|        2|
|eWRfhZUzrAc|Learn the Python ...|2022-08-09T12:54:23Z|    freeCodeCamp.org|     Python for Beginn...|        1|
|bVc8kqh8dKk|🚀 Become a Real ...|2023-02-01T13:00:33Z| Internet Made Coder|     How to MASTER Pyt...|        1|
|Pl9RxjJ5--4|                    |2022-06-10T17:23:37Z|   Coding with Lewis|     How to be a Pytho...|        1|
|YSF1xDhw30w|#filmchina #VSOIn...|2023-02-02T13:00:08Z|       VSO Indonesia|   【INDO SUB】Mutant .